In [7]:
import pandas as pd
import numpy as np
import os

# ---------------------------
# CARGA DE ARCHIVO
# ---------------------------
addresses = [
    'C:/Users/RONALD Q/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/FPDEVANADO.xlsx',
    'C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/FPDEVANADO.xlsx'
]

df = None
for path in addresses:
    if os.path.exists(path):
        df = pd.read_excel(path, header=1)
        print(f"✅ Archivo cargado desde: {path}")
        break

if df is None:
    raise FileNotFoundError("❌ No se encontró el archivo en ninguna de las rutas especificadas.")
df["SERIE"] = df["SERIE"].astype(str)
df['SERIE'] = df['SERIE'].astype(str).str.replace(" ", "")

✅ Archivo cargado desde: C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/FPDEVANADO.xlsx


In [8]:
# ---------------------------
# LIMPIEZA DE DATOS
# ---------------------------
# df["SERIE"] = df["SERIE"].astype(str)
df.columns = df.columns.str.replace(r"\n", " ", regex=True).str.strip()
df = df.dropna(axis=1, how="all")

# Seleccionar columnas que terminan con 'pF'
cols_cd = ["SERIE", "FECHA"] + [c for c in df.columns if c.endswith("pF")]
df_cd = df[cols_cd].copy()

# Aseguramos que FECHA sea datetime
df_cd["FECHA"] = pd.to_datetime(df_cd["FECHA"], errors="coerce")

In [9]:
# ---------------------------
# CÁLCULO DE DELTAS
# ---------------------------
cap_cols = [c for c in df_cd.columns if c.endswith("pF")]

for col in cap_cols:
    ref = df_cd.groupby("SERIE").apply(lambda g: g.loc[g["FECHA"].idxmin(), col])
    ref_mapped = df_cd["SERIE"].map(ref)

    delta = abs((df_cd[col] - ref_mapped) / ref_mapped) * 100
    delta = delta.replace([np.inf, -np.inf], np.nan)

    mask = df_cd[col].isna() | ref_mapped.isna() | (ref_mapped == 0) | (df_cd[col] == 0)
    delta[mask] = np.nan

    df_cd[f"{col}_Delta"] = delta

df_cd['Max_CD'] = df_cd[[c for c in df_cd.columns if c.endswith("Delta")]].max(axis=1)
df_cd.head()

C:\Users\roquispec\AppData\Local\Temp\ipykernel_34800\2362908246.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ref = df_cd.groupby("SERIE").apply(lambda g: g.loc[g["FECHA"].idxmin(), col])
C:\Users\roquispec\AppData\Local\Temp\ipykernel_34800\2362908246.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ref = df_cd.groupby("SERIE").apply(lambda g: g.loc[g["FECHA"].idxmin(), col])
C:\Users\roquispec\AppData\Loc

,SERIE,FECHA,H ICH+ICHL pF,H ICH pF,H ICHL pF,H ICHT pF,X ICL+ICLT pF,X ICL pF,X ICLT pF,X ICLH pF,...,H ICHT pF_Delta,X ICL+ICLT pF_Delta,X ICL pF_Delta,X ICLT pF_Delta,X ICLH pF_Delta,Y ICT+ICTH pF_Delta,Y ICT pF_Delta,Y ICTH pF_Delta,Y ICTL pF_Delta,Max_CD
0,59571,2016-10-23,3802.9,1553.5,2249.5,NaN,4830.8,4820.3,10.40,NaN,...,NaN,NaN,1.626531,NaN,NaN,NaN,NaN,NaN,NaN,76.088964
1,59571,2006-01-29,3786.3,1546.1,2240.2,NaN,4814.2,4803.3,10.15,NaN,...,NaN,NaN,1.973469,NaN,NaN,NaN,NaN,NaN,NaN,76.202863
2,59571,1996-04-16,11300.0,6497.0,2200.0,NaN,NaN,4900.0,NaN,NaN,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
3,59572,2016-09-18,3757.2,1570.5,2185.9,NaN,4711.7,4701.1,9.84,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,59572,2006-01-29,3740.8,1562.9,2176.6,NaN,4688.7,4678.6,10.02,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# ---------------------------
# CONFIGURACIÓN DE PUNTAJE
# ---------------------------
df_cd['CD'] = np.select(
    [
        df_cd['Max_CD'] > 10,
        (df_cd['Max_CD'] > 5) & (df_cd['Max_CD'] <= 10),
        df_cd['Max_CD'] <= 5
    ],
    [5, 3, 1],
    default=np.nan
)
df_cd.head()

,SERIE,FECHA,H ICH+ICHL pF,H ICH pF,H ICHL pF,H ICHT pF,X ICL+ICLT pF,X ICL pF,X ICLT pF,X ICLH pF,...,X ICL+ICLT pF_Delta,X ICL pF_Delta,X ICLT pF_Delta,X ICLH pF_Delta,Y ICT+ICTH pF_Delta,Y ICT pF_Delta,Y ICTH pF_Delta,Y ICTL pF_Delta,Max_CD,CD
0,59571,2016-10-23,3802.9,1553.5,2249.5,NaN,4830.8,4820.3,10.40,NaN,...,NaN,1.626531,NaN,NaN,NaN,NaN,NaN,NaN,76.088964,5.0
1,59571,2006-01-29,3786.3,1546.1,2240.2,NaN,4814.2,4803.3,10.15,NaN,...,NaN,1.973469,NaN,NaN,NaN,NaN,NaN,NaN,76.202863,5.0
2,59571,1996-04-16,11300.0,6497.0,2200.0,NaN,NaN,4900.0,NaN,NaN,...,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.0
3,59572,2016-09-18,3757.2,1570.5,2185.9,NaN,4711.7,4701.1,9.84,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,59572,2006-01-29,3740.8,1562.9,2176.6,NaN,4688.7,4678.6,10.02,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:

# ---------------------------
# TABLAS BASE
# ---------------------------
df_full = df_cd.drop(columns='Max_CD')               # detallada (SERIE, FECHA, pF, Delta, CD)
df_CD = df_cd[['SERIE', 'FECHA', 'CD']].copy()       # original (SERIE, FECHA, CD)

# ---------------------------
# EXTENSIÓN DEL CALENDARIO DESDE 2025
# ---------------------------
inicio = "2015-01-01"
desde_2025 = f"{pd.Timestamp.today().year}-01-01"
fecha_inicio = pd.Timestamp(inicio)  # en el 2026 cambiar ---****
fecha_fin = pd.Timestamp.today().normalize()
fechas = pd.date_range(fecha_inicio, fecha_fin, freq="D")

todas_series = df['SERIE'].dropna().unique()
calendario = pd.MultiIndex.from_product([todas_series, fechas], names=["SERIE", "FECHA"])
df_calendario = pd.DataFrame(index=calendario).reset_index()

# ---------- Tabla extendida CD ----------
ultimos_2024 = df_CD[df_CD['FECHA'] < fecha_inicio].sort_values('FECHA').groupby('SERIE').tail(1)
ultimos_2024['FECHA'] = fecha_inicio
base_ext = pd.concat([df_CD, ultimos_2024], ignore_index=True)

df_extendida_CD = pd.merge(df_calendario, base_ext, on=["SERIE","FECHA"], how="left")
df_extendida_CD = df_extendida_CD.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)

# ---------- Tabla detallada extendida ----------
ultimos_2024_det = df_full[df_full['FECHA'] < fecha_inicio].sort_values('FECHA').groupby('SERIE').tail(1)
ultimos_2024_det['FECHA'] = fecha_inicio
base_ext_det = pd.concat([df_full, ultimos_2024_det], ignore_index=True)

df_detalles_ext_CD = pd.merge(df_calendario, base_ext_det, on=["SERIE","FECHA"], how="left")
df_detalles_ext_CD = df_detalles_ext_CD.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)


C:\Users\roquispec\AppData\Local\Temp\ipykernel_34800\3884623132.py:26: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_extendida_CD = df_extendida_CD.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)
C:\Users\roquispec\AppData\Local\Temp\ipykernel_34800\3884623132.py:34: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_detalles_ext_CD = df_detalles_ext_CD.groupby("SERIE").apply(lambda g: g.ffill())

In [12]:
# ---------------------------
# FUNCIONES PARA LLAMAR
# ---------------------------
def get_df_CD():
    return df_CD

def get_df_extendida_CD():
    return df_extendida_CD

def get_df_detalles_CD():
    # Quitar columnas que terminan en Delta
    cols = [c for c in df_full.columns if not c.endswith('Delta')]
    # Reordenar: SERIE, FECHA, CD, ...resto
    base = ['SERIE', 'FECHA', 'CD']
    resto = [c for c in cols if c not in base]
    orden = base + resto
    return df_full[orden]

def get_df_detalles_ext_CD():
    # Quitar columnas que terminan en Delta
    cols = [c for c in df_detalles_ext_CD.columns if not c.endswith('Delta')]
    # Reordenar: SERIE, FECHA, CD, ...resto
    base = ['SERIE', 'FECHA', 'CD']
    resto = [c for c in cols if c not in base]
    orden = base + resto
    return df_detalles_ext_CD[orden]

# ---------------------------
# DEMO DE RESULTADOS
# ---------------------------
print("\n ====== TABLA CD ORIGINAL ====== \n")
print(get_df_CD(), "\n")

print("\n ====== TABLA CD EXTENDIDA ====== \n")
print(get_df_extendida_CD().head(), "\n")

print("\n ====== TABLA DETALLADA CD ====== \n")
print(get_df_detalles_CD().head(), "\n")

print("\n ====== TABLA DETALLADA EXTENDIDA CD ====== \n")
print(get_df_detalles_ext_CD().tail(), "\n")


 ====== TABLA CD ORIGINAL ====== 

            SERIE      FECHA   CD
0           59571 2016-10-23  5.0
1           59571 2006-01-29  5.0
2           59571 1996-04-16  1.0
3           59572 2016-09-18  NaN
4           59572 2006-01-29  NaN
..            ...        ...  ...
383  5337PA196-03 2025-01-19  NaN
384       D518294 2025-06-05  NaN
385       D518294 2025-06-05  NaN
386        359111 2022-12-14  NaN
387       L-30230 2023-11-02  NaN

[388 rows x 3 columns] 


 ====== TABLA CD EXTENDIDA ====== 

    SERIE      FECHA  CD
0  100138 2015-01-01 NaN
1  100138 2015-01-02 NaN
2  100138 2015-01-03 NaN
3  100138 2015-01-04 NaN
4  100138 2015-01-05 NaN 


 ====== TABLA DETALLADA CD ====== 

   SERIE      FECHA   CD  H ICH+ICHL pF  H ICH pF  H ICHL pF  H ICHT pF  \
0  59571 2016-10-23  5.0         3802.9    1553.5     2249.5        NaN   
1  59571 2006-01-29  5.0         3786.3    1546.1     2240.2        NaN   
2  59571 1996-04-16  1.0        11300.0    6497.0     2200.0        NaN   
3  5